This is my attempt at an SVM from scratch. It does not work on mnist as it cannot work on multi class classification. It works for the dataset used in week 7.

In [6]:
import numpy as np
import keras
import pickle
import tensorflow as tf

with open(f"./Models/CNN/localBest.pkl", "rb") as f:
  model = pickle.load(f)

FileNotFoundError: ignored

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import cv2
from google.colab.patches import cv_imshow
import glob
from skimage.feature import hog

In [3]:
%cd /content/drive/MyDrive/Msc Artificial Intelligence/Semester 1/Machine Learning and Computer Vision/Assginment

/content/drive/MyDrive/Msc Artificial Intelligence/Semester 1/Machine Learning and Computer Vision/Assginment


In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
humanDataset = [cv2.imread(filepath) for filepath in glob.glob("/content/drive/MyDrive/Msc Artificial Intelligence/Semester 1/Machine Learning and Computer Vision/Week 7 SVM/dataset/pedestrians_positive/*.ppm")]

In [ ]:
humanDataset = [cv2.cvtColor(image, cv2.COLOR_BGR2GRAY) for image in humanDataset]

In [ ]:
#Your code goes here
nonHumanDataset = [cv2.imread(filepath) for filepath in glob.glob("/content/drive/MyDrive/Msc Artificial Intelligence/Semester 1/Machine Learning and Computer Vision/Week 7 SVM/dataset/pedestrians_negative/*.jpg")]

In [ ]:
nonHumanDataset = [cv2.cvtColor(image, cv2.COLOR_BGR2GRAY) for image in nonHumanDataset]
nonHumanDataset = [cv2.resize(image, (64, 128)) for image in nonHumanDataset]

nonHumanDatasetNorm = [image / 255 for image in nonHumanDataset]
humanDatasetNorm = [image / 255 for image in humanDataset]

hogFeaturesForNonHuman = [hog(image, 9, (8, 8)) for image in nonHumanDatasetNorm]

hogFeatureForHuman = [hog(image, 9, (8, 8)) for image in humanDatasetNorm]

In [ ]:
nonHumanDatasetCopy = [cv2.imread(filepath) for filepath in glob.glob("/content/drive/MyDrive/Msc Artificial Intelligence/Semester 1/Machine Learning and Computer Vision/Week 7 SVM/dataset/pedestrians_negative/*.jpg")]

In [ ]:
def generateNegativeData (X, samplesWanted=100):

  samples = []

  h = 128

  w = 64

  for i in range(samplesWanted):

    image = X[np.random.randint(len(X)-1)]

    shapeX, shapeY, shapeZ = np.shape(image)

    x = np.random.randint(shapeX - h)

    y = np.random.randint(shapeY - w)

    samples.append(image[x : x + h, y : y+w, :])

  return samples



In [ ]:
negativeImages = generateNegativeData(nonHumanDatasetCopy, 1000)

In [ ]:
negativeImages = [cv2.cvtColor(image, cv2.COLOR_BGR2GRAY) for image in negativeImages]

negativeImages = [image / 255 for image in negativeImages]

hogNegativeImages = [hog(image, 9, (8, 8)) for image in negativeImages]

In [ ]:
yNonHuman = [0] * 1000
yHuman = [1] * 924

y = yHuman + yNonHuman

In [ ]:
X = hogFeatureForHuman + hogNegativeImages

In [ ]:
from sklearn.model_selection import train_test_split

xTrain, xTest, yTrain, yTest = train_test_split(X, y, random_state=42)

In [ ]:
def NormVector (vector):
  return 0.5 * np.dot(vector.T, vector)

In [ ]:
def HingeLoss (y, yPred):
  result = y * yPred
  return 0 if result > 1 else 1 - result

In [ ]:
def CostFunction(W, y, yPred):

  #norm = np.linalg.norm(W)
  norm = NormVector(W)
  #print(f"Norm: {norm}")
  cost = HingeLoss(y, yPred)
  #print(f'cost: {cost}')
  return norm + cost

In [ ]:
def DecisionFuncion(W, X, b):
  return np.dot(W.T, X) + b

In [ ]:
def GradientDescent(X, y, W, epochs=10):

  bias = 0
  weights = W
  wiggle = 0.001
  eta = 0.00001

  for i in range(epochs):

    meanCost = 0.0
    predictions = []
    gradients = [0.0] * 6804
    for j, data in enumerate(X):
      yPred = DecisionFuncion(weights, data, bias)

      cost = 0
      if(y[j] == 0):
        cost = CostFunction(weights, -1, yPred)
        costWiggle = CostFunction(weights+wiggle, -1, yPred)
      else:
        cost = CostFunction(weights, y[j], yPred)
        costWiggle = CostFunction(weights+wiggle, y[j], yPred)


      predictions.append(yPred)
      meanCost += cost
      gradient = (cost - costWiggle) / wiggle

      #print(f"wiggle: {cost} | {costWiggle}")
      #print(f"gradient: {gradient}")

      gradients.append(gradient)

    meanCost = meanCost / len(X)
    meanGradient = np.mean(gradients)

    for t, weight in enumerate(weights):
      weights[t] -= -eta*meanGradient

    print(f"Loss is: {meanCost}")

  return weights

In [ ]:
W = np.random.rand(6804)
print(W)

[0.60664158 0.46099618 0.33720343 ... 0.12780573 0.28667485 0.71942752]


In [ ]:
weights = GradientDescent(xTrain, yTrain, W, 1000)

Loss is: 1288.0223223061562
Loss is: 1266.0406978170276
Loss is: 1244.5591229106599
Loss is: 1223.5660029021067
Loss is: 1203.0500146695874
Loss is: 1183.0001002609263
Loss is: 1163.405460651213
Loss is: 1144.2555496476966
Loss is: 1125.540067938431
Loss is: 1107.2489572816255
Loss is: 1089.37239483189
Loss is: 1071.9007876005066
Loss is: 1054.824767046561
Loss is: 1038.13518379536
Loss is: 1021.8231024820499
Loss is: 1005.8797967162226
Loss is: 990.2967441659744
Loss is: 975.0656217576231
Loss is: 960.1783009886271
Loss is: 945.6268433514779
Loss is: 931.4034958650159
Loss is: 917.5006867117032
Loss is: 903.9110209774258
Loss is: 890.6272764920403
Loss is: 877.6423997679278
Loss is: 864.9495020343684
Loss is: 852.5418553654597
Loss is: 840.4128888993718
Loss is: 828.5561851468207
Loss is: 816.9654763865307
Loss is: 805.6346411456719
Loss is: 794.5577007634923
Loss is: 783.7288160357848
Loss is: 773.1422839385295
Loss is: 762.792534428896
Loss is: 752.6741273214907
Loss is: 742.7817492

In [ ]:
def predict(X, y, W):
  accuracy = 0.0

  correct = 0

  for i, data in enumerate(X):
    modelDecision = DecisionFuncion(W, data, 0.0)
    #print(f"My Decision: {modelDecision}")
    #print(f"Label: {y[i]}")

    decision = 0 if modelDecision < 0 else 1
    if(y[i] == decision):
      correct += 1


  print(f"accuracy = {correct / len(X)}")

In [ ]:
predict(xTest, yTest, weights)

accuracy = 0.7234927234927235
